In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision.datasets import FashionMNIST
from torchvision.transforms import ToTensor

device = "cuda" if torch.cuda.is_available() else "cpu"

/usr/local/lib/python3.8/dist-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def train(dataloader, model, loss_fn, optimizer):
    model.train()  # set the model to training mode
    n_samples = len(dataloader.dataset)  # the num of samples
    for batch_i, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)
        preds = model(X)
        loss = loss_fn(preds, y)
        optimizer.zero_grad()  # init grads to 0
        loss.backward()
        optimizer.step()
        if batch_i % 100 == 0:
            loss, prog = loss.item(), batch_i * len(X)
            print(f"Loss: {loss:>7f}, Progress: {prog:>5d}/{n_samples:>5d}")


def test(dataloader, model, loss_fn):
    model.eval()  # set the model to evaluation mode
    n_samples = len(dataloader.dataset)
    test_loss, correct = 0, 0
    with torch.no_grad():  # fix grads
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            preds = model(X)
            loss = loss_fn(preds, y)
            test_loss += loss.item()
            correct += (preds.argmax(1) == y).type(torch.float).sum().item()
    acc = correct / n_samples
    avg_loss = test_loss / n_samples
    print(f"[Test] acc: {acc:>7f}, avg_loss: {avg_loss:>7f}")

In [3]:
class NeuralNet(nn.Module):

    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_stack = nn.Sequential(
            nn.Linear(28*28, 512),  # img size: 28x28
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10))  # classes: 10
    
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_stack(x)
        return logits

In [4]:
# model
model = NeuralNet().to(device)

# hyper parameters
lr = 1e-3  # learning rate, 0.001
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
epochs = 5
batch_size = 64

# data
training_data = FashionMNIST(root="data",train=True, download=True, transform=ToTensor())
test_data = FashionMNIST(root="data", train=False, download=True, transform=ToTensor())

# data loaders
train_dataloader = DataLoader(training_data, batch_size)
test_dataloader = DataLoader(test_data, batch_size)

100%|██████████| 26421880/26421880 [00:12<00:00, 2056700.35it/s]


Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 29515/29515 [00:00<00:00, 52584.25it/s]


Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 4422102/4422102 [00:04<00:00, 918718.85it/s] 


Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 5148/5148 [00:00<00:00, 11316707.02it/s]

Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw



In [5]:
for epoch in range(epochs):
    print(f"Epoch {epoch+1} -----")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done.")

Epoch 1 -----
Loss: 2.302204, Progress:     0/60000
Loss: 0.569436, Progress:  6400/60000
Loss: 0.391492, Progress: 12800/60000
Loss: 0.486012, Progress: 19200/60000
Loss: 0.436846, Progress: 25600/60000
Loss: 0.438789, Progress: 32000/60000
Loss: 0.371465, Progress: 38400/60000
Loss: 0.524198, Progress: 44800/60000
Loss: 0.491258, Progress: 51200/60000
Loss: 0.496055, Progress: 57600/60000
[Test] acc: 0.845500, avg_loss: 0.006703
Epoch 2 -----
Loss: 0.266615, Progress:     0/60000
Loss: 0.337058, Progress:  6400/60000
Loss: 0.279586, Progress: 12800/60000
Loss: 0.380013, Progress: 19200/60000
Loss: 0.435605, Progress: 25600/60000
Loss: 0.391259, Progress: 32000/60000
Loss: 0.305290, Progress: 38400/60000
Loss: 0.474170, Progress: 44800/60000
Loss: 0.419834, Progress: 51200/60000
Loss: 0.423865, Progress: 57600/60000
[Test] acc: 0.857600, avg_loss: 0.006131
Epoch 3 -----
Loss: 0.230002, Progress:     0/60000
Loss: 0.313785, Progress:  6400/60000
Loss: 0.229860, Progress: 12800/60000
Lo

In [6]:
pth = "0_quickstart.pth"
torch.save(model.state_dict(), pth)
print(f"Saved model: {pth}")

Saved model: 0_quickstart.pth


In [7]:
model = NeuralNet()
model.load_state_dict(torch.load(pth))

classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",]

model.eval()
X, y = test_data[0]
with torch.no_grad():
    pred = model(X)
    y_, y = classes[pred[0].argmax(0)], classes[y]
    print(f"Predicted: {y_}, Actual: {y}")

Predicted: Ankle boot, Actual: Ankle boot
